Import Packages

In [3]:
import torch
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

File Path

In [5]:
Audio_train = 'data/npy/Apr9/train_mfcc_fold8_50_delta_delta.npy'
Audio_val = 'data/npy/Apr9/val_mfcc_fold8_50_delta_delta.npy'


Quote: https://colab.research.google.com/drive/134NRf4PAR1CuQejf6q3DTDZNaFklR8NY?usp=sharing

In [17]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate=0.2):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_rate)
        self.relu = nn.ReLU()
        self.cost = nn.CrossEntropyLoss()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

<b>Data Preprocessing <br></b>
Z-Normalization

In [64]:
################# set to your own path ##############
Features_Labels = np.load(Audio_train)
#test_Features = np.load('data/npy/Apr9/test_mfcc13_delta_delta.npy')
Val_Features_Labels = np.load(Audio_val)
################# set to your own path ##############


no_instances = Features_Labels.shape[0]
no_classes = 10
no_features = Features_Labels.shape[1] - no_classes
Features = Features_Labels[:,0:no_features]
Labels_array = Features_Labels[:,no_features:Features_Labels.shape[1]]
Labels = []
for i in Labels_array:
    Labels.append(np.where(i == max(i))[0][0])

Val_Features = Val_Features_Labels[:,0:no_features]
Val_Labels_array = Val_Features_Labels[:,no_features:Features_Labels.shape[1]]
Val_Labels = []
for i in Val_Labels_array:
    Val_Labels.append(np.where(i == max(i))[0][0])
print (Features.shape, Labels_array.shape, Val_Features.shape, Val_Labels_array.shape)#, test_Features.shape)



############### Normalization ###################
mean = np.mean(Features, axis=0, keepdims=True)
std = np.std(Features, axis=0, ddof=1, keepdims=True)

# Z-normalize each row
Features_normalized = (Features - mean) / std

# mean = np.mean(test_Features, axis=1, keepdims=True)
# std = np.std(test_Features, axis=1, ddof=1, keepdims=True)

# Z-normalize each row
# test_Features_norm = (test_Features - mean) / std
# test_Features_norm

Val_Features_norm = (Val_Features - mean) / std
Val_Features_norm
############### Normalization ####################

(800, 300) (800, 10) (100, 300) (100, 10)


array([[-0.70532743,  0.50288148, -0.15561555, ...,  0.71970385,
         0.24446883,  0.03173463],
       [-0.9106413 ,  1.15877521,  0.53727727, ...,  1.76773419,
         1.48372101,  1.84433382],
       [-1.45079868,  0.20518245,  1.25889523, ..., -0.19996044,
        -0.65831403, -0.989708  ],
       ...,
       [ 0.79297733, -0.72589192, -0.84976611, ..., -0.80418456,
        -0.78035322, -0.7266562 ],
       [ 0.58603427, -0.23799363, -0.24808601, ..., -0.377274  ,
        -0.72369873, -0.44429014],
       [-0.60684619, -0.42799432, -0.15083106, ..., -0.54413445,
        -0.3896985 , -0.76947425]])

Build the MLP

In [65]:
#scaler = StandardScaler() # not necessary if you do normalization before

train_dataset = TensorDataset(torch.tensor(Features_normalized, dtype=torch.float32), torch.tensor(Labels, dtype=torch.long))
val_dataset = TensorDataset(torch.tensor(Val_Features_norm, dtype=torch.float32), torch.tensor(Val_Labels, dtype=torch.long))

batch_size = 40
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = batch_size)

# input layer
# number of features
input_size = Features_normalized.shape[1]
# number of hidden layer's nodes
hidden_size = 64
output_size = Labels_array.shape[1]

model = MLP(input_size, hidden_size, output_size)
print (model)

optimizer = optim.Adam(model.parameters(), lr = 0.001, weight_decay = 0.01)


MLP(
  (fc1): Linear(in_features=300, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (relu): ReLU()
  (cost): CrossEntropyLoss()
)


Train the MLP

In [67]:
Epochs = 10
for ep in range(Epochs):
    model.train()
    train_loss = 0.0
    correct_train = 0
    total_train = 0
    for inputs, targets in train_loader:
        # remove the gradient from the preious batch to remove bias
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = model.cost(outputs,targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total_train += targets.size(0)
        correct_train += (predicted == targets).sum().item()

    train_acc = correct_train/total_train
    print (f"Epoch{ep+1}: Train loss: {train_loss/len(train_loader):.4f}, Train Acc: {train_acc: .2f}")
    
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            loss = model.cost(outputs, targets)
            val_loss += loss.item()
            _,predicted = torch.max(outputs, 1)
            total_val += targets.size(0)
            correct_val += (predicted == targets).sum().item()

    val_acc = correct_val/total_val
    print (f"Epoch{ep+1}: Val loss: {val_loss/len(val_loader):.4f}, Val Acc: {val_acc: .2f}")
    

Epoch1: Train loss: 1.5049, Train Acc:  0.50
Epoch1: Val loss: 1.4949, Val Acc:  0.51
Epoch2: Train loss: 1.2636, Train Acc:  0.61
Epoch2: Val loss: 1.3562, Val Acc:  0.61
Epoch3: Train loss: 1.0883, Train Acc:  0.66
Epoch3: Val loss: 1.2580, Val Acc:  0.66
Epoch4: Train loss: 0.9550, Train Acc:  0.72
Epoch4: Val loss: 1.1689, Val Acc:  0.70
Epoch5: Train loss: 0.8523, Train Acc:  0.75
Epoch5: Val loss: 1.1055, Val Acc:  0.72
Epoch6: Train loss: 0.7574, Train Acc:  0.78
Epoch6: Val loss: 1.0629, Val Acc:  0.72
Epoch7: Train loss: 0.7069, Train Acc:  0.81
Epoch7: Val loss: 1.0066, Val Acc:  0.75
Epoch8: Train loss: 0.6355, Train Acc:  0.83
Epoch8: Val loss: 0.9830, Val Acc:  0.74
Epoch9: Train loss: 0.5801, Train Acc:  0.86
Epoch9: Val loss: 0.9564, Val Acc:  0.74
Epoch10: Train loss: 0.5275, Train Acc:  0.88
Epoch10: Val loss: 0.9385, Val Acc:  0.76


In [63]:
len(Labels)

800

In [57]:
outputs[0]

tensor([ 0.5370,  0.4016,  0.0627, -0.0926, -0.1298,  0.4039, -0.7045,  0.2633,
        -0.2386, -0.2109])

In [48]:
model.cost(outputs[0],targets[0])

RuntimeError: Expected floating point type for target with class probabilities, got Long